In [1]:
import numpy as np
import pandas as pd
import os
import cv2

In [2]:
classified_image=["train","test"]
categories=["angry","disgust","fear","happy","neutral","sad","surprise"]
category_index={"angry":0,"disgust":1,"fear":2,"happy":3,"neutral":4,"sad":5,"surprise":6}

In [3]:
x_train=[]    #the values of trained images are stored in this array
y_train=[]    #the indexes of trained images that are beloged to their category are stored in this array
x_test=[]     #the values of tested images are stored in this array
y_test=[]     #the indexes of that tested images that are beloged to their category are stored in this array

In [4]:
data_path="C:\\Users\\pasum\\Desktop\\Emotion Detector\\archive"

In [5]:
def img_resizing(img_path):
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=cv2.resize(img,(48,48))
    return img

In [6]:
for classify_img in classified_image:
    
    path1=os.path.join(data_path,classify_img)   #path1=C:\Users\pasum\Desktop\Emotion Detector\archive\....(i.e test or train)
    print("Extracting the "+classify_img+" images/.......")
    
    for category in categories:
        folder_path=os.path.join(path1,category) #folder_path=C:\Users\pasum\Desktop\Emotion Detector\archive\(i.e test or train)
        images=os.listdir(folder_path)
        
        print(f'    loading images of the category {category} in {classify_img} images.......')
        
        for image in images:
            img_path=os.path.join(folder_path,image)
            try:
                resized_img=img_resizing(img_path)
                if classify_img=="train":
                    x_train.append(resized_img)
                    y_train.append(category_index[category])
                else:
                    x_test.append(resized_img)
                    y_test.append(category_index[category])
                    
            
            except:
                pass

print("Fetching data completed........")

Extracting the train images/.......
    loading images of the category angry in train images.......
    loading images of the category disgust in train images.......
    loading images of the category fear in train images.......
    loading images of the category happy in train images.......
    loading images of the category neutral in train images.......
    loading images of the category sad in train images.......
    loading images of the category surprise in train images.......
Extracting the test images/.......
    loading images of the category angry in test images.......
    loading images of the category disgust in test images.......
    loading images of the category fear in test images.......
    loading images of the category happy in test images.......
    loading images of the category neutral in test images.......
    loading images of the category sad in test images.......
    loading images of the category surprise in test images.......
Fetching data completed........


In [7]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [8]:
x_train.shape

(28709, 48, 48)

In [9]:
y_train.shape

(28709,)

In [10]:
x_test.shape

(7178, 48, 48)

In [11]:
y_test.shape

(7178,)

In [12]:
#there are 7 emotions so..
numOfCategories=7

In [13]:
from keras.utils import to_categorical

In [14]:
#Althogh when we converted to grayscale the imgs still have 3 color channels(BGR),so we have to RESHAPE it
x_train=np.reshape(x_train,(x_train.shape[0],48,48,1))
x_test=np.reshape(x_test,(x_test.shape[0],48,48,1))

In [15]:
y_train=to_categorical(y_train,numOfCategories)
y_test=to_categorical(y_test,numOfCategories)

In [16]:
x_train.shape

(28709, 48, 48, 1)

In [17]:
y_train.shape

(28709, 7)

In [18]:
x_test.shape

(7178, 48, 48, 1)

In [19]:
y_test.shape

(7178, 7)

In [20]:
import keras

In [21]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers.normalization import BatchNormalization

In [23]:
model=Sequential()

model.add(Conv2D(64,kernel_size=(3,3),input_shape=(48,48,1),activation="elu"))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=(3,3),input_shape=(48,48,1),activation="elu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,kernel_size=(3,3),activation="elu"))
model.add(BatchNormalization())
model.add(Conv2D(128,kernel_size=(3,3),activation="elu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256,kernel_size=(3,3),activation="elu"))
model.add(BatchNormalization())
model.add(Conv2D(256,kernel_size=(3,3),activation="elu"))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))



model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(7,activation="softmax"))


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 44, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       7

In [25]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [42]:
model.fit(x_train,y_train,batch_size=50,epochs=20,validation_data=(x_test,y_test))

Epoch 1/20
575/575 [==============================] - 341s 594ms/step - loss: 1.6184 - accuracy: 0.3558 - val_loss: 2.9630 - val_accuracy: 0.3050
Epoch 2/20
575/575 [==============================] - 347s 604ms/step - loss: 1.3676 - accuracy: 0.4717 - val_loss: 1.2706 - val_accuracy: 0.5183
Epoch 3/20
575/575 [==============================] - 392s 681ms/step - loss: 1.2511 - accuracy: 0.5237 - val_loss: 1.3485 - val_accuracy: 0.4876
Epoch 4/20
575/575 [==============================] - 375s 652ms/step - loss: 1.1752 - accuracy: 0.5549 - val_loss: 1.1845 - val_accuracy: 0.5573
Epoch 5/20
575/575 [==============================] - 359s 624ms/step - loss: 1.1166 - accuracy: 0.5778 - val_loss: 1.1834 - val_accuracy: 0.5571
Epoch 6/20
575/575 [==============================] - 363s 631ms/step - loss: 1.0635 - accuracy: 0.6010 - val_loss: 1.0732 - val_accuracy: 0.5995
Epoch 7/20
575/575 [==============================] - 355s 618ms/step - loss: 1.0117 - accuracy: 0.6191 - val_loss: 1.1064 -

# Saving the model


In [3]:
# serialize to JSON
json_file = model.to_json()
with open("json_file_path", "w") as file:
   file.write(json_file)
# serialize weights to HDF5
model.save_weights("h5_file")

NameError: name 'model' is not defined